In [1]:
import os
import random
import math
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
import pandas as pd

In [2]:
base_dir = '/home/ubuntu/kaggle/flowers/xray_images/Data/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [3]:
"""
HyperParameters:
- Learning Rate 
- Momentum 
- Number of Conv Blocks
- Number of Layers in Each Block 
- Parameters of each Layer (Conv, Pooling, Flatten, Dense)
- Regularization 
- Batch Normalization 

"""

'\nHyperParameters:\n- Learning Rate \n- Momentum \n- Number of Conv Blocks\n- Number of Layers in Each Block \n- Parameters of each Layer (Conv, Pooling, Flatten, Dense)\n- Regularization \n- Batch Normalization \n\n'

In [4]:
def Conv2D_Input(filters, kernel_size, strides, padding, activation, kernel_regularizer, name, input_shape):
    return tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                  strides = strides, padding = padding, activation = activation, 
                                  kernel_regularizer = kernel_regularizer, name = name, 
                                  input_shape = input_shape)


def Conv2D(filters, kernel_size, strides, padding, activation, kernel_regularizer, name):
    return tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                  strides = strides, padding = padding, activation = activation, 
                                  kernel_regularizer = kernel_regularizer, name = name)

def MaxPool(pool_size, strides, padding, name):
    return tf.keras.layers.MaxPooling2D(pool_size = pool_size, strides = strides, 
                                        padding = padding, name = name)

def Flatten(name):
    return tf.keras.layers.Flatten(name = name)

def Dense(units, activation, kernel_regularizer, name):
    return tf.keras.layers.Dense(units = units, activation = activation, 
                                 kernel_regularizer=kernel_regularizer,name = name)

def Batch_Normalize():
    return tf.keras.layers.BatchNormalization(axis=1)

def DropOut(dropout_rate):
    return tf.keras.layers.Dropout(dropout_rate)

In [16]:
# Note - To be convereted to functions before posting
def set_hyperparameters(param):
    
    LR_10P_Range = param['LR_10P_Range'] 
    Mtm_Val = param['Mtm_Val'] 
    BS_Val = param['BS_Val']  
    Cv_Bks_Val= param['Cv_Bks_Val']  
    Cv_CvLy_Val= param['Cv_CvLy_Val']  
    Cv_PoLy_Val = param['Cv_PoLy_Val'] 
    Cv_KS_Val = param['Cv_KS_Val']  
    Cv_Reg_Val = param['Cv_Reg_Val']
    Cv_Fil_Val = param['Cv_Fil_Val']  
    Cv_Pd_Val = param['Cv_Pd_Val']  
    Cv_Sd_Val = param['Cv_Sd_Val']  
    Cv_Av_Val = param['Cv_Av_Val']  
    Po_PS_Val = param['Po_PS_Val'] 
    Po_Pd_Val = param['Po_Pd_Val'] 
    Dl_Num_Val = param['Dl_Num_Val']  
    Dl_Unt_Val = param['Dl_Unt_Val']  
    Dl_Reg_Val = param['Dl_Reg_Val']  
    Dl_Reg_Val = param['Dl_Reg_Val'] 
    Bch_Norm_Val = param['Bch_Norm_Val']  
    Dpout_Val = param['Dpout_Val']  
    Dp_Rate = param['Dp_Rate']
    Opt_Algo_Val= param['Opt_Algo_Val']  
    Input_Shape = param['Input_Shape']
    

    LR = pow(10,random.uniform(LR_10P_Range[0],LR_10P_Range[1]))
    
    random.shuffle(Mtm_Val)
    Mtm = Mtm_Val[0]
    
    random.shuffle(BS_Val)
    BS = BS_Val[0]
    
    random.shuffle(Cv_Bks_Val)
    Cv_Bks = Cv_Bks_Val[0]
    
    Cv_CvLy = []
    Cv_KS = []
    Cv_Reg = []
    Po_PS = []
    Po_Sd = []

    random.shuffle(Cv_KS_Val)
    random.shuffle(Cv_Reg_Val)
    random.shuffle(Po_PS_Val)
    random.shuffle(Cv_CvLy_Val)
    
    for i in range(Cv_Bks):
        Cv_CvLy.append(Cv_CvLy_Val[0])
        Cv_KS.append(Cv_KS_Val[0])
        Cv_Reg.append(Cv_Reg_Val[0])
        Po_PS.append(Po_PS_Val[0])
        Po_Sd.append(Po_PS_Val[0])

    Cv_Fil = []    
    random.shuffle(Cv_Fil_Val)

    for i in range(Cv_Bks):
        Cv_Fil.append(Cv_Fil_Val[0]*pow(2,math.ceil((i+1)/2)-1))  
        
    Cv_Pd = []
    Cv_Sd = []
    Cv_Av = []
    Cv_PoLy = []
    Po_Pd = []

    for i in range(Cv_Bks):
        Cv_Pd.append(Cv_Pd_Val)
        Cv_Sd.append(Cv_Sd_Val)
        Cv_Av.append(Cv_Av_Val)
        Cv_PoLy.append(Cv_PoLy_Val)
        Po_Pd.append(Po_Pd_Val)
    
    random.shuffle(Dl_Num_Val)
    Dl_Num = Dl_Num_Val[0]
    
    Dl_Unt =[]

    for i in range(Dl_Num):
        random.shuffle(Dl_Unt_Val)
        Dl_Unt.append(Dl_Unt_Val[0])
        
    Dl_Reg = []
    random.shuffle(Dl_Reg_Val)

    for i in range(Dl_Num):
        Dl_Reg.append(Dl_Reg_Val[0])
    
    Dl_Av = []

    for i in range(Dl_Num):
        Dl_Av.append(Dl_Av_Val)

    random.shuffle(Bch_Norm_Val)
    Bch_Norm = Bch_Norm_Val[0]
    
    random.shuffle(Dpout_Val)
    Dpout = Dpout_Val[0]
    
    random.shuffle(Opt_Algo_Val)
    Opt_Algo = Opt_Algo_Val[0]
    
    hyperparams = {'LR':LR, 'Mtm':Mtm, 'BS':BS,
                    'Cv_Bks':Cv_Bks, 'Cv_CvLy':Cv_CvLy, 'Cv_PoLy':Cv_PoLy, 
                    'Cv_KS':Cv_KS, 'Cv_Reg':Cv_Reg, 'Cv_Fil':Cv_Fil, 
                    'Cv_Pd':Cv_Pd, 'Cv_Sd':Cv_Sd, 'Cv_Av':Cv_Av, 
                    'Po_PS':Po_PS, 'Po_Pd':Po_Pd, 'Po_Sd':Po_Sd,
                    'Dl_Num':Dl_Num,'Dl_Unt':Dl_Unt, 'Dl_Reg':Dl_Reg, 'Dl_Av':Dl_Av, 
                    'Bch_Norm':Bch_Norm, 'Dpout':Dpout, 'Dp_Rate':Dp_Rate,
                    'Opt_Algo':Opt_Algo, 'Input_Shape':Input_Shape}
    
    return hyperparams

In [17]:
def generate_model(hyperparams):
    
    Cv_Bks  = hyperparams['Cv_Bks']
    Cv_CvLy = hyperparams['Cv_CvLy']
    Cv_PoLy = hyperparams['Cv_PoLy']
    Cv_KS = hyperparams['Cv_KS']
    Cv_Reg = hyperparams['Cv_Reg']
    Cv_Fil = hyperparams['Cv_Fil']
    Cv_Pd = hyperparams['Cv_Pd']
    Cv_Sd = hyperparams['Cv_Sd']
    Cv_Av = hyperparams['Cv_Av']
    Po_PS = hyperparams['Po_PS']
    Po_Pd = hyperparams['Po_Pd']
    Po_Sd = hyperparams['Po_Sd']
    Dl_Num = hyperparams['Dl_Num']
    Dl_Unt = hyperparams['Dl_Unt']
    Dl_Reg = hyperparams['Dl_Reg']
    Dl_Av = hyperparams['Dl_Av']
    Bch_Norm = hyperparams['Bch_Norm']
    Dpout = hyperparams['Dpout']
    Dp_Rate = hyperparams['Dp_Rate']
    
    Model = tf.keras.Sequential()
    
    for i in range(Cv_Bks):

        Cv_Bk_Name = 'Blk' + str(i+1)

        for j in range(Cv_CvLy[i]):
            if (i==0 and j==0):
                
                Ly_Name = Cv_Bk_Name + '_Conv' + str(j+1)
                Ly = Conv2D_Input(Cv_Fil[i], Cv_KS[i],
                                  Cv_Sd[i], Cv_Pd[i],
                                  Cv_Av[i], Cv_Reg[i],                         
                                  Ly_Name, Input_Shape)
                Model.add(Ly)

                if Bch_Norm == 'conv' or Bch_Norm == 'all':
                    Ly = Batch_Normalize()
                    Model.add(Ly)

                if Dpout == 'conv' or Dpout == 'all':
                    Ly = DropOut(Dp_Rate)
                    Model.add(Ly)

            else:
     

                Ly_Name = Cv_Bk_Name + '_Conv' + str(j+1)
                Ly = Conv2D(Cv_Fil[i], Cv_KS[i],
                            Cv_Sd[i], Cv_Pd[i],
                            Cv_Av[i], Cv_Reg[i],                         
                            Ly_Name)
                Model.add(Ly)
            
                if Bch_Norm == 'conv' or Bch_Norm == 'all':
                    Ly = Batch_Normalize()
                    Model.add(Ly)

                if Dpout == 'conv' or Dpout == 'all':
                    Ly = DropOut(Dp_Rate)
                    Model.add(Ly)

        for k in range(Cv_PoLy[i]):

            Ly_Name = Cv_Bk_Name + '_Pool' + str(k+1)
            Ly = MaxPool(Po_PS[i], Po_Sd[i],
                         Po_Pd[i], Ly_Name)
            Model.add(Ly)
        
    Model.add(Flatten('Flatten'))

    for l in range(Dl_Num):
        
        Ly_Name = 'Dense' + str(l+1)

        Ly = Dense(Dl_Unt[l], Dl_Av[l], Dl_Reg[l], Ly_Name)
        Model.add(Ly)

        if Bch_Norm == 'dense' or Bch_Norm == 'all':
            Ly = Batch_Normalize()
            Model.add(Ly)

        if Dpout == 'dense' or Dpout == 'all':
            Ly = DropOut(Dp_Rate)
            Model.add(Ly)

    Model.add(Dense(1, 'sigmoid', None, 'Output'))
    
    return Model

In [18]:
def prepare_callbacks(es_patience):
    callback_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=es_patience,
                                                               verbose=1, restore_best_weights=True)
    #callback_model= ModelCheckpoint(filepath='/home/ubuntu/kaggle/flowers/xray_images/Outputs/Best_Model', 
                                    #save_best_only=True, save_weights_only=True)
    callbacks = [callback_early_stopping] #, callback_model]
    return callbacks

In [19]:
def compile_and_run_model(Model, Opt_Algo, LR, Mtm, Loss, Metrics):
    
    if Opt_Algo == 'sgd':
        Optimizer = tf.keras.optimizers.SGD(LR, Mtm)
        Model.compile(optimizer = Optimizer, loss = Loss, metrics = Metrics)
    
    elif Opt_Algo == 'adam':
        Optimizer = tf.keras.optimizers.Adam(learning_rate = LR)
        Model.compile(optimizer = Optimizer, loss = Loss, metrics = Metrics)

In [20]:
def prepare_train_test_data(train_dir, test_dir, batch_size, input_size):
    
    train_datagen = ImageDataGenerator(rescale=1.0/255.)
    test_datagen = ImageDataGenerator(rescale=1.0/255.)
    
    train_gen = train_datagen.flow_from_directory(train_dir,batch_size=batch_size,color_mode= 'rgb',
                                                    class_mode='binary',target_size=input_size) 
    test_gen = test_datagen.flow_from_directory(test_dir,batch_size=batch_size,color_mode= 'rgb',
                                                    class_mode='binary',target_size=input_size)
    
    return train_gen, test_gen

In [21]:
def export_results(iter_number, history, hyperparams, File_Name1, File_Name2):
    
    file = open(File_Name1,"w")
    file.write(str(hyperparams))
    file.close()
    
    df = pd.DataFrame.from_dict(history.history.items())
    
    df0 = pd.DataFrame()
    for i in range(7):
        df1 = pd.DataFrame(df[1][i],columns = [df[0][i]])
        df0 = pd.concat([df0, df1], axis=1)
    df0.to_csv(File_Name2)
    
    print("Files for iteration number "+ str(iter_number) + " have been exported.")

In [22]:
Input_Shape = (224,224,3) 
Input_Shape_WO_Channel = (224,224) 
Loss = 'binary_crossentropy'
Metrics = ['accuracy','Precision','Recall']

In [23]:
L2 = tf.keras.regularizers.l2(0.005)
Dp_Rate = 0.5

In [38]:
LR_10P_Range = [-5,-2] # Learning Rate 
Mtm_Val = [0] # Momentum 
BS_Val = [16,32,64,128] # Batch Size

Cv_Bks_Val = [4,5] # Number of Conv Blocks
Cv_CvLy_Val = [1,2] # Num of Conv Layers by Conv Blocks (Can be different in each Conv Block)
Cv_PoLy_Val = 1 # Num of Pooling layers by Conv Blocks (Same by each Conv Block)

Cv_KS_Val = [3] # Kernerl Size in Conv Layers by Conv Blocks (Same across all Conv Blocks) 
Cv_Reg_Val = [None] # Regularizaion Type in Conv Layers by Conv Blocks (Same across all Conv Blocks)
Cv_Fil_Val = [16,24] # Number of Filters in Conv Layers by Conv Blocks (This fixes the first, rest are followed a-a-2a-2a-4a)
Cv_Pd_Val = 'same'
Cv_Sd_Val = (1,1)
Cv_Av_Val = 'relu'

Po_PS_Val = [(2,2)] # Pool Size in Pool Layers by Conv Blocks 
Po_Pd_Val = 'same'

Dl_Num_Val = [1]
Dl_Unt_Val = [128,256,512,1024]
Dl_Reg_Val = [None]
Dl_Av_Val = 'relu'

Bch_Norm_Val = ['none']
Dpout_Val = ['none']
Opt_Algo_Val = ['sgd','adam']

param_values = {'LR_10P_Range':LR_10P_Range, 'Mtm_Val':Mtm_Val, 'BS_Val':BS_Val,
                'Cv_Bks_Val':Cv_Bks_Val, 'Cv_CvLy_Val':Cv_CvLy_Val, 'Cv_PoLy_Val':Cv_PoLy_Val, 
                'Cv_KS_Val':Cv_KS_Val, 'Cv_Reg_Val':Cv_Reg_Val, 
                'Cv_Fil_Val':Cv_Fil_Val, 'Cv_Pd_Val':Cv_Pd_Val, 
                'Cv_Sd_Val':Cv_Sd_Val, 'Cv_Av_Val':Cv_Av_Val, 
                'Po_PS_Val':Po_PS_Val, 'Po_Pd_Val':Po_Pd_Val, 
                'Dl_Num_Val':Dl_Num_Val, 'Dl_Unt_Val':Dl_Unt_Val,
                'Dl_Reg_Val':Dl_Reg_Val, 'Dl_Av_Val':Dl_Av_Val, 
                'Bch_Norm_Val':Bch_Norm_Val, 'Dpout_Val':Dpout_Val,
                'Dp_Rate':Dp_Rate, 'Opt_Algo_Val':Opt_Algo_Val,
                'Input_Shape':Input_Shape}

In [39]:
os.chdir('/home/ubuntu/kaggle/flowers/xray_images/Output_Iter1')

In [15]:
for i in range(25):
    
    hyperparams = set_hyperparameters(param_values)
    Model = generate_model(hyperparams)
    
    Model.summary()

    print("")
    print("Batch_Size: "+str(hyperparams['BS']))
    print("Learning_Rate: "+str(hyperparams['LR']))
    print("Momentum: "+str(hyperparams['Mtm']))
    print("Optimization_Algo: "+str(hyperparams['Opt_Algo']))
    print("Regularization_Cv_Layers" + str(hyperparams['Cv_Reg']))
    print("Regularization_Dense_Layers" + str(hyperparams['Dl_Reg']))
    print ()
    print("")

    train_gen, test_gen = prepare_train_test_data(train_dir, test_dir, 
                                                      hyperparams['BS'], Input_Shape_WO_Channel)
    compile_and_run_model(Model, hyperparams['Opt_Algo'], hyperparams['LR'], hyperparams['Mtm'],
                              Loss, Metrics)
    callbacks = prepare_callbacks(6)
    
    history = Model.fit(train_gen, validation_data= test_gen,
                            epochs = 30, verbose=1, callbacks = callbacks)
    
    File_Name1 = 'Hyperparams_Model_' + str(i) + '.txt'
    File_Name2 = 'Model_Results_' + str(i) + '.csv'
    
    export_results(i, history, hyperparams, File_Name1, File_Name2)
    del(Model, history, train_gen, test_gen, hyperparams)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Blk1_Conv1 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
Blk1_Conv2 (Conv2D)          (None, 224, 224, 32)      9248      
_________________________________________________________________
Blk1_Pool1 (MaxPooling2D)    (None, 112, 112, 32)      0         
_________________________________________________________________
Blk2_Conv1 (Conv2D)          (None, 112, 112, 32)      9248      
_________________________________________________________________
Blk2_Conv2 (Conv2D)          (None, 112, 112, 32)      9248      
_________________________________________________________________
Blk2_Pool1 (MaxPooling2D)    (None, 56, 56, 32)        0         
_________________________________________

Epoch 1/20
41/41 [==============================] - 59s 1s/step - loss: 2.0973 - acc: 0.5326 - precision_2: 0.7547 - recall_2: 0.5494 - val_loss: 1.7586 - val_acc: 0.6250 - val_precision_2: 0.6250 - val_recall_2: 1.0000
Epoch 2/20
41/41 [==============================] - 52s 1s/step - loss: 1.7307 - acc: 0.6043 - precision_2: 0.7426 - recall_2: 0.7154 - val_loss: 1.7575 - val_acc: 0.6250 - val_precision_2: 0.6250 - val_recall_2: 1.0000
Epoch 3/20
41/41 [==============================] - 53s 1s/step - loss: 1.7184 - acc: 0.6271 - precision_2: 0.7393 - recall_2: 0.7693 - val_loss: 1.7564 - val_acc: 0.6250 - val_precision_2: 0.6250 - val_recall_2: 1.0000
Epoch 4/20
41/41 [==============================] - 53s 1s/step - loss: 1.7029 - acc: 0.6561 - precision_2: 0.7421 - recall_2: 0.8230 - val_loss: 1.7553 - val_acc: 0.6250 - val_precision_2: 0.6250 - val_recall_2: 1.0000
Epoch 5/20
41/41 [==============================] - 53s 1s/step - loss: 1.7019 - acc: 0.6729 - precision_2: 0.7453 - rec

Epoch 1/20
41/41 [==============================] - 59s 1s/step - loss: 4.8676 - acc: 0.4492 - precision_4: 0.7350 - recall_4: 0.4044 - val_loss: 3.6691 - val_acc: 0.3750 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 2/20
41/41 [==============================] - 52s 1s/step - loss: 4.1627 - acc: 0.6093 - precision_4: 0.7502 - recall_4: 0.7107 - val_loss: 3.6755 - val_acc: 0.3750 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 3/20
41/41 [==============================] - 53s 1s/step - loss: 4.1195 - acc: 0.6256 - precision_4: 0.7438 - recall_4: 0.7566 - val_loss: 3.6786 - val_acc: 0.3750 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 4/20
41/41 [==============================] - 53s 1s/step - loss: 4.0728 - acc: 0.6432 - precision_4: 0.7548 - recall_4: 0.7698 - val_loss: 3.6794 - val_acc: 0.3750 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 5/20
41/41 [==============================] - 53s 1s/step - loss: 4.0429 - acc: 0.

Epoch 6/20
82/82 [==============================] - 54s 654ms/step - loss: 0.2414 - acc: 0.9732 - precision_8: 0.9802 - recall_8: 0.9837 - val_loss: 6.7412 - val_acc: 0.6250 - val_precision_8: 0.6250 - val_recall_8: 1.0000
Epoch 7/20
82/82 [==============================] - 54s 661ms/step - loss: 0.1679 - acc: 0.9732 - precision_8: 0.9812 - recall_8: 0.9827 - val_loss: 4.3204 - val_acc: 0.6266 - val_precision_8: 0.6260 - val_recall_8: 1.0000
Epoch 00007: early stopping
Files for iteration number 4 have been exported.
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Blk1_Conv1 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
dropout_23 (Dropout)         (None, 224, 224, 32)      0         
_________________________________________________________________
Blk1_Conv2 (Conv2D)          (None, 224, 224, 32)      9248

Epoch 00007: early stopping
Files for iteration number 5 have been exported.
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Blk1_Conv1 (Conv2D)          (None, 224, 224, 32)      2432      
_________________________________________________________________
dropout_33 (Dropout)         (None, 224, 224, 32)      0         
_________________________________________________________________
Blk1_Conv2 (Conv2D)          (None, 224, 224, 32)      25632     
_________________________________________________________________
dropout_34 (Dropout)         (None, 224, 224, 32)      0         
_________________________________________________________________
Blk1_Pool1 (MaxPooling2D)    (None, 112, 112, 32)      0         
_________________________________________________________________
Blk2_Conv1 (Conv2D)          (None, 112, 112, 32)      25632     
___________________________________________

...

Epoch 1/20
41/41 [==============================] - 55s 1s/step - loss: 3.0938 - acc: 0.8832 - precision_40: 0.9687 - recall_40: 0.8710 - val_loss: 2.8790 - val_acc: 0.6250 - val_precision_40: 0.6250 - val_recall_40: 1.0000
Epoch 2/20
41/41 [==============================] - 51s 1s/step - loss: 1.9418 - acc: 0.9664 - precision_40: 0.9805 - recall_40: 0.9742 - val_loss: 2.2080 - val_acc: 0.6250 - val_precision_40: 0.6250 - val_recall_40: 1.0000
Epoch 3/20
41/41 [==============================] - 53s 1s/step - loss: 1.4030 - acc: 0.9676 - precision_40: 0.9776 - recall_40: 0.9788 - val_loss: 1.8467 - val_acc: 0.6250 - val_precision_40: 0.6250 - val_recall_40: 1.0000
Epoch 4/20
41/41 [==============================] - 53s 1s/step - loss: 1.0730 - acc: 0.9758 - precision_40: 0.9827 - recall_40: 0.9848 - val_loss: 1.6529 - val_acc: 0.6250 - val_precision_40: 0.6250 - val_recall_40: 1.0000
Epoch 5/20
41/41 [==============================] - 53s 1s/step - loss: 0.8723 - acc: 0.9806 - precision

Epoch 1/20
41/41 [==============================] - 60s 1s/step - loss: 2.0827 - acc: 0.7314 - precision_42: 0.7435 - recall_42: 0.9747 - val_loss: 2.0955 - val_acc: 0.6250 - val_precision_42: 0.6250 - val_recall_42: 1.0000
Epoch 2/20
41/41 [==============================] - 50s 1s/step - loss: 1.9722 - acc: 0.7429 - precision_42: 0.7429 - recall_42: 1.0000 - val_loss: 2.0344 - val_acc: 0.6250 - val_precision_42: 0.6250 - val_recall_42: 1.0000
Epoch 3/20
41/41 [==============================] - 53s 1s/step - loss: 1.9148 - acc: 0.7429 - precision_42: 0.7429 - recall_42: 1.0000 - val_loss: 1.9771 - val_acc: 0.6250 - val_precision_42: 0.6250 - val_recall_42: 1.0000
Epoch 4/20
41/41 [==============================] - 53s 1s/step - loss: 1.8640 - acc: 0.7429 - precision_42: 0.7429 - recall_42: 1.0000 - val_loss: 1.9218 - val_acc: 0.6250 - val_precision_42: 0.6250 - val_recall_42: 1.0000
Epoch 5/20
41/41 [==============================] - 54s 1s/step - loss: 1.8082 - acc: 0.7429 - precision

Epoch 1/20
82/82 [==============================] - 57s 701ms/step - loss: 7.7814 - acc: 0.7015 - precision_44: 0.7925 - recall_44: 0.8103 - val_loss: 7.3196 - val_acc: 0.3750 - val_precision_44: 0.0000e+00 - val_recall_44: 0.0000e+00
Epoch 2/20
82/82 [==============================] - 52s 637ms/step - loss: 6.4474 - acc: 0.8441 - precision_44: 0.8756 - recall_44: 0.9210 - val_loss: 6.5890 - val_acc: 0.3750 - val_precision_44: 0.0000e+00 - val_recall_44: 0.0000e+00
Epoch 3/20
82/82 [==============================] - 53s 649ms/step - loss: 5.6825 - acc: 0.8992 - precision_44: 0.9181 - recall_44: 0.9489 - val_loss: 5.7983 - val_acc: 0.6250 - val_precision_44: 0.6250 - val_recall_44: 1.0000
Epoch 4/20
82/82 [==============================] - 54s 655ms/step - loss: 5.0737 - acc: 0.9281 - precision_44: 0.9467 - recall_44: 0.9572 - val_loss: 5.4162 - val_acc: 0.6250 - val_precision_44: 0.6250 - val_recall_44: 1.0000
Epoch 5/20
82/82 [==============================] - 53s 651ms/step - loss: 4

Epoch 1/20
326/326 [==============================] - 65s 198ms/step - loss: 3.0663 - acc: 0.7161 - precision_46: 0.8854 - recall_46: 0.7097 - val_loss: 3.0137 - val_acc: 0.6330 - val_precision_46: 0.6300 - val_recall_46: 1.0000
Epoch 2/20
326/326 [==============================] - 55s 169ms/step - loss: 2.7027 - acc: 0.8760 - precision_46: 0.9441 - recall_46: 0.8854 - val_loss: 3.4531 - val_acc: 0.6250 - val_precision_46: 0.6250 - val_recall_46: 1.0000
Epoch 3/20
326/326 [==============================] - 55s 170ms/step - loss: 2.5453 - acc: 0.9164 - precision_46: 0.9531 - recall_46: 0.9334 - val_loss: 3.7420 - val_acc: 0.6346 - val_precision_46: 0.6311 - val_recall_46: 1.0000
Epoch 4/20
326/326 [==============================] - 55s 170ms/step - loss: 2.4370 - acc: 0.9229 - precision_46: 0.9509 - recall_46: 0.9450 - val_loss: 4.0990 - val_acc: 0.6282 - val_precision_46: 0.6270 - val_recall_46: 1.0000
Epoch 5/20
326/326 [==============================] - 55s 169ms/step - loss: 2.3302 

Epoch 1/20
163/163 [==============================] - 61s 375ms/step - loss: 1.7329 - acc: 0.4858 - precision_48: 0.7333 - recall_48: 0.4839 - val_loss: 2.6064 - val_acc: 0.3750 - val_precision_48: 0.0000e+00 - val_recall_48: 0.0000e+00
Epoch 2/20
163/163 [==============================] - 53s 323ms/step - loss: 1.6371 - acc: 0.5391 - precision_48: 0.7798 - recall_48: 0.5290 - val_loss: 2.2124 - val_acc: 0.3750 - val_precision_48: 0.0000e+00 - val_recall_48: 0.0000e+00
Epoch 3/20
163/163 [==============================] - 53s 325ms/step - loss: 1.5975 - acc: 0.5729 - precision_48: 0.8049 - recall_48: 0.5610 - val_loss: 1.8190 - val_acc: 0.3750 - val_precision_48: 0.0000e+00 - val_recall_48: 0.0000e+00
Epoch 4/20
163/163 [==============================] - 53s 324ms/step - loss: 1.5700 - acc: 0.5995 - precision_48: 0.8293 - recall_48: 0.5804 - val_loss: 1.6475 - val_acc: 0.3750 - val_precision_48: 0.0000e+00 - val_recall_48: 0.0000e+00
Epoch 5/20
163/163 [==============================] 